In [432]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from python.mappings import FOOTBALL_LEAGUES_MAPPING

%matplotlib inline

In [433]:
# Constant variables
TABLE_SIZE = 1546
TRAIN_RATIO = 0.8

In [434]:
# Joins train and test datasets together to simplify the process
def concat_df(df1, df2):
    return pd.concat([df1, df2], ignore_index=True)

# Divides the overall dataset into train and test 
def divide_df(df):
    train_size = int(TABLE_SIZE*TRAIN_RATIO)
    
    return df.loc[:train_size - 1], df.loc[train_size:] 

In [435]:
train_data = pd.read_csv('data/train_stats.csv')
test_data = pd.read_csv('data/test_stats.csv')
all_data = concat_df(train_data, test_data)

**1. PREPARATION STEP: Cleaning the data**

In [436]:
all_data.drop(['Player_URL', 'Team_URL'], axis=1, inplace=True)

In [437]:
# Cleans the columns containing commas 
def clean_comma(column):
    column = column.strip()
    column = column[:column.find(',')] + '' + column[column.find(',') + 1:]
    return column

# Cleans the columns containing '\t' symbol
def clean_tab(column):
    return column.strip().replace('\t', '')

In [438]:
comma_columns = ['Team', 'Position']
tab_columns = ['Goals', 'Assists', 'Yel', 'Red', 'SpG', 'AerialsWon', 'MotM', 'Tackles',
               'Inter', 'Fouls', 'Offsides', 'Clear', 'Drb_x', 'Blocks', 'OwnG', 'KeyP',
               'Drb_y', 'Fouled', 'Off', 'Disp', 'UnsTch', 'Crosses', 'LongB', 'ThrB']

for column in comma_columns:
    all_data[column] = all_data[column].apply(clean_comma)
    
for column in tab_columns:
    all_data[column] = all_data[column].apply(clean_tab)

In [439]:
# Some columns with integer values contain symbol '-' instead of 0. The function fixes it
def remove_dashes(column):
    if column == '-':
        return 0
    return column

In [440]:
dash_columns = ['Goals', 'Assists', 'Yel', 'Red', 'SpG', 'AerialsWon', 'MotM', 'Tackles',
                'Inter', 'Fouls', 'Offsides', 'Clear', 'Drb_x', 'Drb_y', 'Blocks', 'OwnG', 'KeyP',
                'Fouled', 'Off', 'Disp', 'UnsTch', 'Crosses', 'LongB', 'ThrB']

for column in dash_columns:
    all_data[column] = all_data[column].apply(remove_dashes)

In [441]:
# Now we have 'Value' variable containing market values in thousands or millions, but we want to get just an integer
def value_scaling(value):
    value = value.strip()
    
    if value.endswith('k'):
        return int(float(value[value.find('€')+1:value.find('k')]) * 10**3)
    elif value.endswith('m'):
        return int(float(value[value.find('€')+1:value.find('m')]) * 10**6)

In [442]:
all_data['Value'] = all_data['Value'].apply(value_scaling)

In [443]:
# We can divide 'Apps' variable because the number of appearances in starting squad is demonstrated in parentheses 
def appearances_division(apps):
    return apps[:apps.find('(')], apps[apps.find('(')+1:apps.find(')')]

In [444]:
all_data['Overall_Apps'], all_data['Start_Apps'] = zip(*all_data['Apps'].apply(appearances_division))

In [445]:
all_data.drop(['Apps'], axis=1, inplace=True)

In [446]:
# It would be better if we replace 'Forward', 'Midfielder' with their short forms (FW, M)

#all_data['Position'].value_counts()

def position_mapping(position):
    position = position.strip()
    
    if position == 'Forward': 
        return 'FW'
    elif position == 'Midfielder': 
        return 'M(C)'
    return position

all_data['Position'] = all_data['Position'].apply(position_mapping)

In [447]:
# We can divide 'Position' column into two positions (if some player has only one position, he`ll get NaN value for the second one).
def position_division(position):
    
    # only two main positions, that`s enough
    if position.count(',') >= 2:
        while position.count(',') != 1:
            position = position[:position.rfind(',')]
          
    if position.find(',') != -1:
        return pd.Series([position[:position.find(',')].strip(), position[position.find(',')+1:]]).values
    return pd.Series([position.strip(), np.nan]).values

# Center (C), Right (R) or Left (L)? This information is contained in parentheses
def position_side(position):
    
    # np.nan has 'float' type
    if type(position) != float:
        if position.find('(') != -1:
            sides = tuple(position[position.find('(')+1:position.find(')')])
            position = position[:position.find('(')].strip(),
            
            return position + sides + tuple([np.nan] * (3 - len(sides)))
        else:
            position = position.strip(),

            return position + tuple([np.nan] * 3)
    else:
        return tuple([np.nan] * 4)

In [448]:
all_data['Position_1'], all_data['Position_2'] = zip(*all_data['Position'].apply(position_division))

In [449]:
all_data['Position_1'], all_data['Side_11'], all_data['Side_12'], all_data['Side_13'] = zip(*all_data['Position_1'].apply(position_side))
all_data['Position_2'], all_data['Side_21'], all_data['Side_22'], all_data['Side_23'] = zip(*all_data['Position_2'].apply(position_side))

In [450]:
all_data.drop(['Position'], axis=1, inplace=True)

In [451]:
# Time to change column types
float_type = ['SpG', 'AerialsWon', 'Tackles', 'Inter', 'Fouls', 'Offsides',
              'Clear', 'Drb_x', 'Blocks', 'KeyP', 'Drb_y', 'Fouled', 'Off',
              'Disp', 'UnsTch', 'Crosses', 'LongB', 'ThrB']
int_type = ['Goals', 'Assists', 'Yel', 'Red', 'MotM', 'OwnG', 'Overall_Apps', 'Start_Apps']

all_data[float_type] = all_data[float_type].astype('float')
all_data[int_type] = all_data[int_type].astype('int64')

In [452]:
# Insert 'Value' variable at the end of the dataset
value_column = all_data.pop('Value')
all_data['Value'] = value_column

# Insert 'Position_2' variable before the variable 'Side_21'
pos2_column = all_data.pop('Position_2')
index = all_data.columns.get_loc('Side_21')
all_data.insert(index, 'Position_2', pos2_column)

**GLOSSARY. DESCRIPTION OF THE VARIABLES**

- SpG - Shots per game
- PS(%) - Percentage of successful passes
- AerialsWon, Aerial - Header in a direct contest with an opponent
- MotM - Man of the Match
- Tackle - Dispossessing an opponent, whether the tackling player comes away with the ball or not
- Interception (Inter) - Preventing an opponent's pass from reaching their teammates
- Fouls - How often a player commits a foul
  Fouled - How often a player gets fouled
- Offsides - It`s referred to "offside won" - the last man to step up to catch an opponent in an offside position
  Off - How often a player gets in offside position
- Clearance (Clear) - Action by a defending player that temporarily removes the attacking threat on their goal/that effectively alleviate pressure on their goal
- Drb_x - How often a player gets dribbled (being dribbled past by an opponent without winning a tackle)
  Drb_y - Frequency of successful dribbles
- Blocks - The number of blocked shots per game
- OwnG - Own goal
- KeyP - Key passes (the final pass leading to a shot at goal from a teammate)
- Dispossessed (Disp) - How often a player gets tackled by an opponent without attempting to dribble past them
- UnsTch - ???
- Average Passes (AvgP) - Average number of passes attempted (short passes, long balls, through balls, crosses)
- Cross - An attempted/accurate pass from a wide position to a central attacking area
- Long Ball (LongB) - An attempted/accurate pass of 25 yards or more
- Through Ball (ThrB) - An attempted/accurate pass between opposition players in their defensive line to find an onrushing teammate (running through on goal)
- Apps - appearances on the football field

In [453]:
#all_data.info()

We see that there are a lot of missing values for 'Side_2X' and 'Side_13' variables (>80%). It describes where a player is located on a football pinch (left, right or center). But it can be useful to create a variable that shows how much place some player "occupies", on how many sides he is able to play. If it isn`t shown (all three variables are NaN values), then we will set 1.

In [454]:
def position_sides(s1, s2, s3):
    sides_list = [s1, s2, s3]
    
    while np.nan in sides_list:
        sides_list.remove(np.nan)
    
    if len(sides_list) == 0:
        return 1
    return len(sides_list)

In [455]:
all_data['Position_1_Sides'] = all_data.apply(lambda x: position_sides(x['Side_11'], x['Side_12'], x['Side_13']), axis=1)
all_data['Position_2_Sides'] = all_data.apply(lambda x: position_sides(x['Side_21'], x['Side_22'], x['Side_23']), axis=1)

Other variables have no missing values so we can continue in research and exploratory analysis

**2. EXPLORATORY DATA ANALYSIS AND FEATURE ENGINEERING**

'Team' variable can be very useful: we can extract a league in which a football player competes, and it is somehow influences his market value. For instance, we expect that in Premier League players cost more because of higher level of this championship and some other aspects. For this task we will use our mapping

In [456]:
all_data['League'] = all_data['Team'].map(FOOTBALL_LEAGUES_MAPPING)

In [463]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1546 entries, 0 to 1545
Data columns (total 45 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              1546 non-null   object 
 1   Team              1546 non-null   object 
 2   Age               1546 non-null   int64  
 3   Mins              1546 non-null   int64  
 4   Goals             1546 non-null   int64  
 5   Assists           1546 non-null   int64  
 6   Yel               1546 non-null   int64  
 7   Red               1546 non-null   int64  
 8   SpG               1546 non-null   float64
 9   PS                1546 non-null   float64
 10  AerialsWon        1546 non-null   float64
 11  MotM              1546 non-null   int64  
 12  Rating            1546 non-null   float64
 13  Tackles           1546 non-null   float64
 14  Inter             1546 non-null   float64
 15  Fouls             1546 non-null   float64
 16  Offsides          1546 non-null   float64


In [459]:
all_data['Team'].unique()

array(['Nice', 'Cagliari', 'Sheff Utd', 'Borussia M.Gladbach', 'Inter',
       'Wolfsburg', 'Darmstadt', 'Nantes', 'Monaco', 'Valencia',
       'Toulouse', 'Union Berlin', 'Brighton', 'Almeria', 'Roma',
       'Atletico', 'Wolves', 'Fulham', 'Reims', 'Lecce', 'Genoa',
       'Osasuna', 'Crystal Palace', 'Man Utd', 'Frosinone',
       'Rayo Vallecano', 'Napoli', 'Empoli', 'FC Koln', 'Clermont Foot',
       'Arsenal', 'Bochum', 'Villarreal', 'Nottingham Forest',
       'Liverpool', 'Leverkusen', 'Marseille', 'Lyon', 'Chelsea',
       'Granada', 'Real Betis', 'Mallorca', 'Deportivo Alaves',
       'Juventus', 'AC Milan', 'Udinese', 'Celta Vigo', 'Brentford',
       'Brest', 'Bayern', 'Torino', 'Atalanta', 'Fiorentina',
       'Borussia Dortmund', 'Tottenham', 'Strasbourg', 'Bologna',
       'Aston Villa', 'Luton', 'Lille', 'Man City', 'Stuttgart',
       'Real Madrid', 'Newcastle', 'Hoffenheim', 'FC Heidenheim', 'PSG',
       'Metz', 'Lazio', 'Real Sociedad', 'West Ham', 'Freiburg', 'Vero

In [460]:
all_data.sample(10)

,Name,Team,Age,Mins,Goals,Assists,Yel,Red,SpG,PS,...,Side_12,Side_13,Position_2,Side_21,Side_22,Side_23,Value,Position_1_Sides,Position_2_Sides,League
29,Jens Cajuste,Napoli,24,1009,0,2,7,0,0.6,86.0,...,NaN,NaN,NaN,NaN,NaN,NaN,10000000,1,1,Serie A
1122,Lloyd Kelly,Bournemouth,25,1558,0,1,2,0,0.2,77.6,...,L,NaN,NaN,NaN,NaN,NaN,18000000,2,1,Premier League
785,Florian Lejeune,Rayo Vallecano,33,3328,3,1,5,0,1.8,81.3,...,NaN,NaN,NaN,NaN,NaN,NaN,3000000,1,1,LaLiga
1442,Marc Roca,Real Betis,27,1988,2,2,5,0,0.8,85.5,...,NaN,NaN,DMC,NaN,NaN,NaN,9000000,1,1,LaLiga
1097,Andreas Christensen,Barcelona,28,1999,2,2,4,0,0.4,94.3,...,NaN,NaN,DMC,NaN,NaN,NaN,40000000,1,1,LaLiga
2,William Osula,Sheff Utd,20,785,0,0,4,0,1.0,53.3,...,NaN,NaN,NaN,NaN,NaN,NaN,3000000,1,1,Premier League
599,Anthony Jung,Werder Bremen,32,2584,1,0,4,1,0.2,87.8,...,L,NaN,M,L,NaN,NaN,1000000,2,1,Bundesliga
504,Mads Pedersen,Augsburg,27,1784,1,1,5,1,0.5,81.3,...,R,NaN,M,C,L,NaN,2500000,2,2,Bundesliga
921,Tommy Doyle,Wolves,22,1216,0,0,2,0,0.5,86.4,...,NaN,NaN,NaN,NaN,NaN,NaN,10000000,1,1,Premier League
1432,Nicolas Pallois,Nantes,36,1586,0,1,5,0,0.2,81.4,...,NaN,NaN,NaN,NaN,NaN,NaN,600000,1,1,Ligue 1


In [461]:
all_data.columns

Index(['Name', 'Team', 'Age', 'Mins', 'Goals', 'Assists', 'Yel', 'Red', 'SpG',
       'PS', 'AerialsWon', 'MotM', 'Rating', 'Tackles', 'Inter', 'Fouls',
       'Offsides', 'Clear', 'Drb_x', 'Blocks', 'OwnG', 'KeyP', 'Drb_y',
       'Fouled', 'Off', 'Disp', 'UnsTch', 'AvgP', 'Crosses', 'LongB', 'ThrB',
       'Overall_Apps', 'Start_Apps', 'Position_1', 'Side_11', 'Side_12',
       'Side_13', 'Position_2', 'Side_21', 'Side_22', 'Side_23', 'Value',
       'Position_1_Sides', 'Position_2_Sides', 'League'],
      dtype='object')

In [462]:
all_data.sample(5)

,Name,Team,Age,Mins,Goals,Assists,Yel,Red,SpG,PS,...,Side_12,Side_13,Position_2,Side_21,Side_22,Side_23,Value,Position_1_Sides,Position_2_Sides,League
203,Lucas Ocampos,Sevilla,29,2887,4,3,9,0,1.8,73.6,...,L,R,FW,NaN,NaN,NaN,9000000,3,1,LaLiga
673,Chiedozie Ogbene,Luton,27,1994,4,1,1,0,1.2,81.6,...,L,R,FW,NaN,NaN,NaN,8000000,3,1,Premier League
620,Paulo Azzi,Cagliari,29,940,0,0,1,0,0.8,71.7,...,NaN,NaN,NaN,NaN,NaN,NaN,1000000,1,1,Serie A
285,Matteo Politano,Napoli,30,2387,8,7,2,1,2.0,83.0,...,L,R,FW,NaN,NaN,NaN,13000000,3,1,Serie A
299,Justin Njinmah,Werder Bremen,23,1013,6,2,1,0,1.4,69.8,...,NaN,NaN,NaN,NaN,NaN,NaN,9000000,1,1,Bundesliga
